In [1]:
# import all the libraries
import pandas as pd 
import numpy as np 
import nltk
import spacy
import matplotlib.pyplot as plt
import os
import random
import requests
import tarfile
from bs4 import BeautifulSoup
from helperfunctions import get_enron_raw_dataset, get_lingspam_dataset

%matplotlib inline

## Data Collection

In [7]:
# Download the Enron Raw enron_emails and parse enron dataset
enron_emails = get_enron_raw_dataset()
lingspam_df = get_lingspam_dataset()

# merge datasets
emails = enron_emails.append(lingspam_df)

file exists
file exists


In [17]:
emails["types"].unique()

array(['ham', 'spam'], dtype=object)